In [2]:
import tensorflow as tf
from keras import layers
from keras import Sequential
import numpy as np
import codecs
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import time
from keras.callbacks import TensorBoard

In [46]:
WORD_LEN = 25
MAX_LEN_CODE = 123.0
END_LEN = 10
TRYING = 1
F_NAME = "data_sets/dict.cc_nouns_with_gender.txt"
FILE_TEST_NAME = "data_sets/deutsch_test_asc_1.txt"
DROP_OUT = 0.2

In [13]:
def readFile(file_name, MAX_LEN_CODE, END_LEN):
    training_data_file = open(file_name, 'r')
    training_data_list = training_data_file.readlines()
    training_data_file.close()

    training_data_list = training_data_list[1:]
    #print(training_data_list[:10])
    np.random.shuffle(training_data_list)
    #print(training_data_list[:10])
    #input()

    f_inputs = [0 for i in range(0, END_LEN)]
    targets = []
    f_inputs = np.asfarray(f_inputs)

    for record in training_data_list:
        all_values = record.split(' ')
        all_values = [word.replace("\n", "") for word in all_values]

        inputs = np.asfarray([ord(letter.lower())/MAX_LEN_CODE for letter in all_values[1]])
        if(len(inputs) > END_LEN):
            inputs = inputs[len(inputs)-END_LEN:]
        elif(len(inputs) < END_LEN):
            begin = np.zeros((END_LEN - len(inputs)), dtype=np.float)
            inputs = np.append(begin, inputs)
        f_inputs = np.vstack((f_inputs, inputs))

        targets.append(all_values[0])

    f_inputs = np.delete(f_inputs, 0, 0)
    encoder = LabelEncoder()
    encoder.fit(targets)
    new_targets = encoder.transform(targets)
    #print(new_targets)
    f_targets = np_utils.to_categorical(new_targets)
    print("Done!")

    return f_inputs, f_targets

In [ ]:
X, Y = readFile(F_NAME, MAX_LEN_CODE, END_LEN)

In [18]:
NAME = "German_noun-8dense-ep100-dataAll-{}".format(int(time.time()))
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

model = Sequential()

model.add(layers.Dense(END_LEN+20, input_dim=END_LEN, activation='relu'))
model.add(layers.Dense(60, activation='relu'))
model.add(layers.Dense(120, activation='relu'))
model.add(layers.Dense(200, activation='relu'))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(15, activation='relu'))
model.add(layers.Dropout(DROP_OUT))
model.add(layers.Dense(3, activation='softmax'))

model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=['accuracy'])

model.fit(X[1000:], Y[1000:], epochs=100, validation_split=0.25, callbacks=[tensorboard])

prediction = model.predict_classes(X[:1000])
np.set_printoptions(threshold=np.nan)
print(prediction)

scores = model.evaluate(X[:1000], Y[:1000], verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))
print(scores)

Train on 280371 samples, validate on 93457 samples
Epoch 1/100
280371/280371 [==============================] - 42s 149us/step - loss: 0.9137 - acc: 0.5815 - val_loss: 0.8550 - val_acc: 0.6180
Epoch 2/100
280371/280371 [==============================] - 44s 158us/step - loss: 0.8278 - acc: 0.6314 - val_loss: 0.7515 - val_acc: 0.6550
Epoch 3/100
280371/280371 [==============================] - 35s 125us/step - loss: 0.7322 - acc: 0.6836 - val_loss: 0.6797 - val_acc: 0.7093
Epoch 4/100
280371/280371 [==============================] - 35s 125us/step - loss: 0.6643 - acc: 0.7218 - val_loss: 0.6066 - val_acc: 0.7502
Epoch 5/100
280371/280371 [==============================] - 34s 120us/step - loss: 0.6175 - acc: 0.7440 - val_loss: 0.5690 - val_acc: 0.7642
Epoch 6/100
280371/280371 [==============================] - 35s 125us/step - loss: 0.5818 - acc: 0.7616 - val_loss: 0.4997 - val_acc: 0.7953
Epoch 7/100
280371/280371 [==============================] - 35s 126us/step - loss: 0.5508 - acc:

Epoch 58/100
280371/280371 [==============================] - 40s 144us/step - loss: 0.2943 - acc: 0.8937 - val_loss: 0.2862 - val_acc: 0.8957
Epoch 59/100
280371/280371 [==============================] - 41s 145us/step - loss: 0.2925 - acc: 0.8939 - val_loss: 0.2662 - val_acc: 0.9033
Epoch 60/100
280371/280371 [==============================] - 35s 125us/step - loss: 0.2929 - acc: 0.8943 - val_loss: 0.2713 - val_acc: 0.9019
Epoch 61/100
280371/280371 [==============================] - 36s 128us/step - loss: 0.2912 - acc: 0.8948 - val_loss: 0.3180 - val_acc: 0.8828
Epoch 62/100
280371/280371 [==============================] - 36s 127us/step - loss: 0.2919 - acc: 0.8950 - val_loss: 0.2933 - val_acc: 0.8931
Epoch 63/100
280371/280371 [==============================] - 36s 127us/step - loss: 0.2895 - acc: 0.8962 - val_loss: 0.2677 - val_acc: 0.9028
Epoch 64/100
280371/280371 [==============================] - 34s 121us/step - loss: 0.2890 - acc: 0.8955 - val_loss: 0.2915 - val_acc: 0.8961

In [ ]:
answer = [list(i).index(1) for i in Y[:1000]]
score = 0
for i in range(len(answer)):
    if answer[i] == prediction[i]:
        score += 1
        pass
    pass
print(score)
print(len(answer))
result = 100 * score/len(answer)
print("{}%".format(round(result)))